# M2 AOS duration test

Using the CSC and the Jupiter Notebook LVV-T1789-M2_aos_duration_test.ipynb select 5 bending modes and it applies each bending mode with positive and negative sign, one by one, holds each bending mode for 39 second and repeats sequence of the bending mode application for 3 times.    


## Import Modules

In [10]:
import asyncio
import yaml
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc

## Declaration of User-defined Functions

In [11]:
def get_bending_mode():
    """Get the bending mode.

    Returns
    -------
    bending_mode : `numpy.ndarray`
        Bending mode.
    """
    bending_mode_file = get_config_dir_ofc() / "M2" / "M2_1um_72_force.yaml"
    with open(bending_mode_file, "r") as yaml_file:
        bending_mode = np.array(yaml.safe_load(yaml_file))

    
    return bending_mode

In [12]:
def get_bending_mode_forces(bending_mode, idx_bending_mode, amplitude):
    """Plot the forces of bending mode.
    
    Parameters
    ----------
    bending_mode : `numpy.ndarray`
        Bending mode data.
    idx_bending_mode : `int`
        Index of bending mode (1-20).
    amplitude : `float`
        Amplitude of bending mode in um.
        
    Returns
    -------
    `numpy.ndarray`
        Actuator forces in Newton.
    """

    # Column 0-2 are the actuator ID, x and y position of actuator
    return amplitude * bending_mode[:, 2+idx_bending_mode]

In [13]:
async def apply_force_cycle_axial(csc, forces, time_forces=5, time_reset=5):
    """Apply the force cycle to axial actuators. The cycle will be positive force, clear,
    negative force, clear.

    Parameters
    ----------
    csc : `lsst.ts.salobj.remote.Remote`
        Remote object of the M2 CSC.
    forces : `numpy.ndarray`
        Axial forces to apply (Newton).
    time_forces : `float`, optional
        Time to apply the forces in second. (the default is 5.0) 
    time_reset : `float`, optional
        Time to reset the forces in second. (the default is 5.0)
    """
    
    # Do the positive direction first
    print(f"Apply the force: {forces} N.")
    await csc.cmd_applyForces.set_start(axial=forces.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

    # Do the Negative direction
    forces_negative = -forces

    print(f"Apply the force: {forces_negative} N.")
    await csc.cmd_applyForces.set_start(axial=forces_negative.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

## Prepare the M2 CSC and put to Enabled state

In [14]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
# get status

state = m2.evt_summaryState.get()
print(state)

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=60)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

## Looping over 5 bending modes 
Loop over 5 bending modes with positive and negative signs for three times. 

In [15]:
bending_mode = get_bending_mode()

In [16]:
# array definition

# bending modes scaling factors
amplitude = np.array([1, 0.1])

In [21]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


idx = 0

for idx in range(1,4):

        # apply 5 bending modes(+/-) in sequence and hold each mode for 39 seconds

        print(f"Status of advancement: Bending mode {5}, mode amplitude {amplitude[0]}")
        forces = get_bending_mode_forces(bending_mode, 5, amplitude[0])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces = 39, time_reset=5)


        print(f"Status of advancement: Bending mode {7}, mode amplitude {amplitude[0]}")
        forces = get_bending_mode_forces(bending_mode, 7, amplitude[0])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces = 39, time_reset=5)


        print(f"Status of advancement: Bending mode {10}, mode amplitude {amplitude[0]}")
        forces = get_bending_mode_forces(bending_mode, 10, amplitude[0])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces = 39, time_reset=5)

        
        print(f"Status of advancement: Bending mode {15}, mode amplitude {amplitude[0]}")
        forces = get_bending_mode_forces(bending_mode, 15, amplitude[0])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces = 39, time_reset=5)
        

        print(f"Status of advancement: Bending mode {20}, mode amplitude {amplitude[1]}")
        forces = get_bending_mode_forces(bending_mode, 20, amplitude[1])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces = 39, time_reset=5)

        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 19:07:37.585102 now.
Status of advancement: Bending mode 5, mode amplitude 1.0
Apply the force: [ 5.026189  5.004667  4.978678  4.933125  4.896649  4.924216  4.896728
  4.933119  4.978629  5.004569  5.026236  5.00468   4.978726  4.93307
  4.896633  4.924184  4.896731  4.933123  4.978682  5.004587  5.026234
  5.004668  4.978679  4.93307   4.89663   4.924216  4.896744  4.933176
  4.978628  5.004577 -1.061852 -1.09986  -1.144153 -1.163315 -1.163287
 -1.144161 -1.099895 -1.061874 -1.061831 -1.099858 -1.144163 -1.163341
 -1.1633   -1.144141 -1.099876 -1.061872 -1.06184  -1.099881 -1.144174
 -1.163321 -1.163281 -1.144137 -1.099885 -1.061897 -6.719647 -6.755349
 -6.799999 -6.807838 -6.80002  -6.755383 -6.719619 -6.755352 -6.800005
 -6.807864 -6.799994 -6.755358 -6.719624 -6.755378 -6.800029 -6.807836
 -6.799994 -6.755359] N.
Reset the force.
Apply the force: [-5.026189 -5.004667 -4.978678 -4.933125 -4.896649 -4.924216 -4.896728
 -4.933119 -4.978629 -5.004569 -5.02623